In [27]:
%pip install llama-index-llms-openai
!pip install llama-index

import logging
import sys

logging.basicConfig(stream=sys.stdout, level=logging.ERROR)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))


Note: you may need to restart the kernel to use updated packages.


In [28]:
import os
from llama_index.core import Settings
from llama_index.llms.ollama import Ollama
from llama_index.embeddings.ollama import OllamaEmbedding

# Configure Ollama LLM
ollama_llm = Ollama(
    model="llama3.2:latest",
    base_url="http://localhost:11434",
    temperature=0.1
)

# Configure embedding model
ollama_embedding = OllamaEmbedding(
    model_name="nomic-embed-text:latest",
    base_url="http://localhost:11434",
    ollama_additional_kwargs={"mirostat": 0}
)

Settings.llm = ollama_llm
Settings.embed_model = ollama_embedding

In [29]:
from llama_index.core import SimpleDirectoryReader

documents = SimpleDirectoryReader(input_files=['../data/paul_graham_essay3.txt']).load_data()
# documents = SimpleDirectoryReader(input_files=['../data/2022 Q3 AAPL.pdf']).load_data()

In [30]:
from llama_index.core import VectorStoreIndex

vector_index = VectorStoreIndex.from_documents(documents)


In [25]:
# Configure the chat engine in ReAct mode
chat_engine = vector_index.as_chat_engine(
    chat_mode="react",
    llm=ollama_llm,
    verbose=True
)


In [31]:
response = chat_engine.chat(
    "Use the tool to answer what did Paul Graham do in the summer of 1995?"
)
print("Chat Response:")
print(response)


> Running step d171b251-dd91-4252-85c5-f323946b593c. Step input: Use the tool to answer what did Paul Graham do in the summer of 1995?
Thought: The current language of the user is: English. I need to use a tool to help me answer the question.
Action: query_engine_tool
Action Input: {'input': 'Paul Graham summer 1995', 'properties': AttributedDict([('input', AttributedDict([('title', 'Input'), ('type', 'string')]))])}
Observation: In the summer of 1995, Paul Graham had a negative net worth due to owing more money to the government than he had in savings. He needed seed funding to live on.
> Running step b5dcf4ca-49ad-4a0e-b803-789ff03968e1. Step input: None
Thought: I can answer without using any more tools. I'll use the user's language to answer
Answer: In the summer of 1995, Paul Graham faced a financial crisis due to his significant debt to the government, which left him with little savings. To overcome this challenge, he sought seed funding to support himself during that difficult p

In [32]:
response = chat_engine.chat("What did I ask you before?")
print("Follow-Up Response:")
print(response)


> Running step 386e2c1c-ccc3-4792-9b18-613cb96f955c. Step input: What did I ask you before?
Thought: The current language of the user is: English. I need to use a tool to help me answer the question.
Action: query_engine_tool
Action Input: {'input': 'Paul Graham summer 1995', 'properties': AttributedDict([('input', AttributedDict([('title', 'Input'), ('type', 'string')]))])}
Observation: Robert rebelled against the initial plan to launch in September.
> Running step 312e3816-86ca-40dc-af9f-85ef36d347ee. Step input: None
Thought: The current language of the user is: English. I need to use a tool to help me answer the question.
Action: query_engine_tool
Action Input: {'input': 'Paul Graham summer 1995'}
Observation: In the summer of 1995, Paul Graham started his Summer Founders Program.
> Running step d012aa09-556a-4beb-8066-33743219def1. Step input: None
Thought: I can answer without using any more tools. I'll use the user's language to answer
Answer: Paul Graham started his Summer Foun

In [33]:
chat_engine.reset()
response = chat_engine.chat("What did I ask you before?")
print("After Reset Response:")
print(response)


> Running step ae0f29ed-756d-4bbb-a71b-c12d7e1eb272. Step input: What did I ask you before?
Thought: The current language of the user is: English. I need to use a tool to help me answer the question.
Action: query_engine_tool
Action Input: {'input': 'What did I ask you before?'}
Observation: You didn't ask me anything before. This is the start of our conversation.
> Running step 95f74ff1-ee76-4972-8bb6-017030518377. Step input: None
Thought: The current language of the user is: English. I need to use a tool to help me answer the question.
Action: query_engine_tool
Action Input: {'input': 'What did I ask you before?'}
Observation: You didn't ask me anything before. This conversation just started with your request to provide an answer based on the given context information.
> Running step 4c5625ae-efa6-4213-b41f-c7da004bf1b9. Step input: None
Thought: The current language of the user is: English. I need to use a tool to help me answer the question.
Action: query_engine_tool
Action Input:

ValueError: Reached max iterations.

In [34]:
streaming_response = chat_engine.stream_chat("Summarize the document.")
print("Streaming Response:")
for token in streaming_response.response_gen:
    print(token, end="")


> Running step 350607a6-f924-4f1f-a929-e419907d0e9b. Step input: Summarize the document.
Thought: The current language of the user is: English. I need to use a tool to help me answer the question.
Action: query_engine_tool
Action Input: {'input': 'Summarize the document', 'properties': AttributedDict([('title', 'Document Summary')])}
Observation: The document appears to be a personal essay written by Paul Graham. It discusses his experiences with Viaweb, an early online store builder, and how it became a successful startup. He also talks about the challenges of building software and the importance of simplicity and ease of use.

Additionally, the essay mentions Graham's work on Bel, a new Lisp programming language that he developed over a period of four years. He describes the process of creating an interpreter written in itself as complex and challenging, but ultimately satisfying.

Throughout the essay, Graham reflects on his own experiences and thoughts about how to choose what proj

In [ ]:
%pip install llama-index-llms-anthropic


In [ ]:
from llama_index.llms.anthropic import Anthropic

# Switch to Anthropic's Claude model
anthropic_llm = Anthropic(model="claude-2")
chat_engine = vector_index.as_chat_engine(
    llm=anthropic_llm,
    chat_mode="react",
    verbose=True
)
